## Day 46 Lecture 1 Assignment

In this assignment, we will calculate a distance matrix for geographical Starbucks data and use it to identify locations that are close together and far apart. We will perform clustering on this dataset later on.

We will be using the "haversine" package to compute geographical distance. It can be pip installed.

In [1]:
!pip install haversine

In [2]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from haversine import haversine

Below is a convenience function for calculating geographical distance matrices using lat-long data.

In [3]:
def geo_sim_matrix(df, col_name = 'Coordinates'):
    """
    A function that computes a geographical distance matrix (in miles).
    Each row in the dataframe should correspond to one location.
    In addition, the dataframe must have a column containing the lat-long of each location as a tuple (i.e. (lat, long)).
    
    Parameters:
        df (pandas dataframe): an nxm dataframe containing the locations to compute similarities between.
        col (string): the name of the column containing the lat-long tuples.
        
    Returns:
        distance (pandas dataframe): an nxn distance matrix between the geographical coordinates of each location.
    """
    
    df = df.copy()
    df.reset_index(inplace=True)
    haver_vec = np.vectorize(haversine, otypes=[np.float32])
    distance = df.groupby('index').apply(lambda x: pd.Series(haver_vec(df[col_name], x[col_name])))
    distance = distance / 1.609344  # converts to miles
    distance.columns = distance.index
    
    return distance


This dataset contains the latitude and longitude (as well as several other details we will not be using) of every Starbucks in the world as of February 2017. Each row consists of the following features, which are generally self-explanatory:

- Brand
- Store Number
- Store Name
- Ownership Type
- Street Address
- City
- State/Province
- Country
- Postcode
- Phone Number
- Timezone
- Longitude
- Latitude

Load in the dataset.

In [6]:
# answer goes here
data = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Data%20Sets%20Clustering/starbucks_locations.csv')
data

,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Postcode,Phone Number,Timezone,Longitude,Latitude
0,Starbucks,47370-257954,"Meritxell, 96",Licensed,"Av. Meritxell, 96",Andorra la Vella,7,AD,AD500,376818720,GMT+1:00 Europe/Andorra,1.53,42.51
1,Starbucks,22331-212325,Ajman Drive Thru,Licensed,"1 Street 69, Al Jarf",Ajman,AJ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,55.47,25.42
2,Starbucks,47089-256771,Dana Mall,Licensed,Sheikh Khalifa Bin Zayed St.,Ajman,AJ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,55.47,25.39
3,Starbucks,22126-218024,Twofour 54,Licensed,Al Salam Street,Abu Dhabi,AZ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,54.38,24.48
4,Starbucks,17127-178586,Al Ain Tower,Licensed,"Khaldiya Area, Abu Dhabi Island",Abu Dhabi,AZ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,54.54,24.51
...,...,...,...,...,...,...,...,...,...,...,...,...,...
25595,Starbucks,21401-212072,Rex,Licensed,"141 Nguyễn Huệ, Quận 1, Góc đường Pasteur và L...",Thành Phố Hồ Chí Minh,SG,VN,70000,08 3824 4668,GMT+000000 Asia/Saigon,106.70,10.78
25596,Starbucks,24010-226985,Panorama,Licensed,"SN-44, Tòa Nhà Panorama, 208 Trần Văn Trà, Quận 7",Thành Phố Hồ Chí Minh,SG,VN,70000,08 5413 8292,GMT+000000 Asia/Saigon,106.71,10.72
25597,Starbucks,47608-253804,Rosebank Mall,Licensed,"Cnr Tyrwhitt and Cradock Avenue, Rosebank",Johannesburg,GT,ZA,2194,27873500159,GMT+000000 Africa/Johannesburg,28.04,-26.15
25598,Starbucks,47640-253809,Menlyn Maine,Licensed,"Shop 61B, Central Square, Cnr Aramist & Coroba...",Menlyn,GT,ZA,181,NaN,GMT+000000 Africa/Johannesburg,28.28,-25.79


Begin by narrowing down the dataset to a specific geographic area of interest. Since we will need to manually compute a distance matrix, which will be on the order of $n^{2}$ in terms of size, we would recommend choosing an area with 3000 or less locations. In this example, we will use Hawaii, which has about 100 locations; for reference, California has about 2800 locations. Feel free to choose a different region that is of more interest to you, if desired.

Subset the dataframe to only include records for Starbucks locations in Hawaii.

In [7]:
# answer goes here
sb = data[data['State/Province'] == 'HI']
sb = sb.drop('index', axis=1)
sb.head()

KeyError: ignored

The haversine package takes tuples with 2 numeric elements and interprets them as lat-long to calculate distance, so add a new column called "Coordinates" that converts the lat and long in each row into a tuple. In other words, the last two columns of the dataframe should initially look like this:

**Latitude, Longitude**  
39.14, -121.64  
34.13, -116.40  
...

After adding the new column, the last three columns should look like this:

**Latitude, Longitude, Coordinates**  
39.14, -121.64,   (39.14, -121.64)  
34.13, -116.40,   (34.13, -116.40)  
...

In [8]:
# answer goes here
sb['Coordinates'] = list(zip(sb['Longitude'], sb['Latitude']))
sb.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Postcode,Phone Number,Timezone,Longitude,Latitude,Coordinates
17202,Starbucks,21034-73360,Aiea Shopping Center,Company Owned,99-115 Aiea Heights Drive #125,Aiea,HI,US,967013913,808-484-1488,GMT-10:00 Pacific/Honolulu,-157.93,21.38,"(-157.93, 21.38)"
17203,Starbucks,21053-99755,Stadium Marketplace,Company Owned,4561 Salt Lake Boulevard,Aiea,HI,US,968183167,808-488-9313,GMT-10:00 Pacific/Honolulu,-157.93,21.37,"(-157.93, 21.37)"
17204,Starbucks,21006-10033,Kaonohi St & Kam Hwy - Pearlridge,Company Owned,98-125 Kaonohi Street,Aiea,HI,US,967012318,808-484-9548,GMT-10:00 Pacific/Honolulu,-157.94,21.38,"(-157.94, 21.38)"
17205,Starbucks,21005-10034,Pearlridge Mall Uptown,Company Owned,98-1005 Moanalua Road,Aiea,HI,US,967014705,808-484-9355,GMT-10:00 Pacific/Honolulu,-157.94,21.39,"(-157.94, 21.39)"
17206,Starbucks,21063-101700,Waimalu Shopping Center,Company Owned,"98-1277 Kaahumanu Street, Building E, Unit 7, ...",Aiea,HI,US,967015314,808-484-5802,GMT-10:00 Pacific/Honolulu,-157.95,21.39,"(-157.95, 21.39)"


Calculate the distance matrix using the starter code/function geo_sim_matrix() provided earlier in the notebook. It assumes the column containing the coordinates for each location is called "Coordinates". Examine the docstring for more details.

Note: the latitude and longitudes provided only go out to two decimal places, which limits the resolution of the distance calculations to about 0.5 miles. Distances that are very small may not be accurately represented here (e.g. several instances of "0 distance" for distinct Starbucks locations in very close proximity).

In [9]:
# answer goes here
sb_ = geo_sim_matrix(sb, col_name = 'Coordinates')
sb_.head()

index,17202,17203,17204,17205,17206,17207,17208,17209,17210,17211,17212,17213,17214,17215,17216,17217,17218,17219,17220,17221,17222,17223,17224,17225,17226,17227,17228,17229,17230,17231,17232,17233,17234,17235,17236,17237,17238,17239,17240,17241,...,17261,17262,17263,17264,17265,17266,17267,17268,17269,17270,17271,17272,17273,17274,17275,17276,17277,17278,17279,17280,17281,17282,17283,17284,17285,17286,17287,17288,17289,17290,17291,17292,17293,17294,17295,17296,17297,17298,17299,17300
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
17202,0.000000,0.640306,0.690934,0.942025,1.523027,6.994635,6.994635,224.446625,225.053207,223.240814,224.446625,9.418554,7.044047,8.476838,3.275129,2.652697,9.418554,10.360241,3.275129,8.476838,9.164237,8.055517,10.474183,2.652697,8.924218,3.275129,3.275129,7.983953,12.180655,4.872692,8.476838,10.474183,1.280613,7.044047,11.598577,3.275129,7.983953,16.492180,6.593312,6.593312,...,107.574745,13.143335,13.143335,173.390945,173.390945,173.794937,178.440826,9.072892,8.677469,9.339848,105.535446,11.509702,11.509702,10.676122,109.848091,109.848091,110.319717,110.503654,172.860031,92.059601,92.059601,91.840874,89.901573,107.138443,105.435257,107.138443,8.057591,7.117158,7.524488,3.046092,115.111580,13.110185,2.436418,17.696173,169.674576,111.935181,104.275177,5.673964,5.800602,6.938969
17203,0.640306,0.000000,0.942025,1.455155,1.884081,6.725522,6.725522,224.146896,224.754303,222.943039,224.146896,8.995823,6.593312,8.055517,2.652697,2.041337,8.995823,9.936370,2.652697,8.055517,8.822593,7.664555,10.095805,2.041337,8.476838,2.652697,2.652697,7.535091,11.856931,4.568851,8.055517,10.095805,0.640306,6.593312,11.258101,2.652697,7.535091,16.254669,6.176124,6.176124,...,107.391739,13.189980,13.189980,172.988373,172.988373,173.390945,178.134369,9.185066,8.887365,9.535160,105.803780,11.348089,11.348089,10.540711,109.617081,109.617081,110.082314,110.692833,172.456177,91.840874,91.840874,91.626060,89.695671,107.368172,105.672554,107.368172,8.479333,7.537062,8.025685,3.365890,114.923065,13.465416,2.169428,17.846395,169.325500,111.686714,104.086296,5.851921,6.177308,7.027100
17204,0.690934,0.942025,0.000000,0.640352,0.942056,6.388272,6.388272,225.057007,225.664078,223.852325,225.057007,9.936663,7.535337,8.996085,3.487028,2.910229,9.936663,10.877410,3.487028,8.996085,9.745002,8.600211,11.029331,2.910229,9.418863,3.487028,3.487028,8.477116,12.773815,5.472629,8.996085,11.029331,1.455155,7.535337,12.180894,3.487028,8.477116,17.131184,7.115760,7.115760,...,108.236481,13.833491,13.833491,173.928162,173.928162,174.330933,179.047287,9.757409,9.339897,10.006135,104.907700,10.847805,10.847805,10.006784,110.492104,110.492104,110.961014,109.843185,173.396149,92.708511,92.708511,92.491302,90.555344,106.493103,104.793098,106.493103,7.537308,6.595037,7.100296,2.436608,115.771530,12.533210,3.045977,17.022440,170.253662,112.571602,104.934967,5.003312,5.238478,6.251312
17205,0.942025,1.455155,0.640352,0.000000,0.690934,6.732244,6.732244,225.357330,225.963547,224.150681,225.357330,10.360579,7.984248,9.418863,4.082803,3.487028,10.360579,11.302264,4.082803,9.418863,10.096039,8.996085,11.412682,3.487028,9.865087,4.082803,4.082803,8.924543,13.106168,5.799852,9.418863,11.412682,2.041466,7.984248,12.529010,4.082803,8.924543,17.380289,7.535337,7.535337,...,108.421822,13.818684,13.818684,174.330933,174.330933,174.735092,179.354446,9.694230,9.185095,9.861807,104.640999,11.053967,11.053967,10.189693,110.724976,110.724976,111.200203,109.656296,173.800171,92.929581,92.929581,92.708511,90.763779,106.265358,104.557747,106.265358,7.117358,6.177477,6.590932,2.169481,115.962173,12.184464,3.365657,16.889206,170.603195,112.821724,105.126198,4.878767,4.873359,6.218408
17206,1.523027,1.884081,0.942056,0.690934,0.000000,6.177646,6.177646,225.967407,226.574112,224.761871,225.967407,10.877734,8.477393,9.936954,4.365464,3.814058,10.877734,11.818634,4.365464,9.936954,10.670853,9.537143,11.964211,3.814058,10.360918,4.365464,4.

For each Starbucks, identify its nearest neighboring location in Hawaii (and presumably in the world). Save the output to a dataframe with three columns: Location, Nearest Neighbor, and Distance (Miles).

In [11]:
# answer goes here
nn = {}
for i in sb_.index:
    distances = sb_[i][sb_.index != i]
    nn[i, distances.idxmin()] = min(distances)

nn = pd.DataFrame(list(nn.items()))
nn['Location'] = nn[0].apply(lambda x: x[0])
nn['Nearest Neighbor'] = nn[0].apply(lambda x: x[1])
nn['Distance in Miles'] = nn[1]
nn.drop(columns=[0,1], inplace=True)
nn.head()

,Location,Nearest Neighbor,Distance in Miles
0,17202,17203,0.640306
1,17203,17202,0.640306
2,17204,17205,0.640352
3,17205,17204,0.640352
4,17206,17205,0.690934


If the nearest neighbor of a Starbucks location is far away, we could consider that Starbucks to be "on an island". Which five Starbucks in Hawaii are the most "on an island"?

In [12]:
# answer goes here
islands = pd.merge(nn,
                   sb[['City', 'Latitude', 'Longitude']],
                   left_on='Nearest Neighbor',
                   right_index=True)

islands = pd.merge(islands,
                   sb[['City', 'State/Province', 'Latitude', 'Longitude']],
                   left_on='Nearest Neighbor',
                   right_index=True)

islands.columns = ['Location', 'Nearest Neighbor', 'Distance in Miles', 
                   'Location City', 'Location Lat', 'Location Long',
                   'NN City', 'NN State', 'NN Lat', 'NN Long']

islands.sort_values(by='Distance in Miles', ascending=False).head()

,Location,Nearest Neighbor,Distance in Miles,Location City,Location Lat,Location Long,NN City,NN State,NN Lat,NN Long
65,17267,17295,16.080042,Waikoloa,19.91,-155.88,Waikoloa,HI,19.91,-155.88
93,17295,17267,16.080042,Kamuela,20.02,-155.67,Kamuela,HI,20.02,-155.67
76,17278,17284,8.027675,Lihue,21.97,-159.38,Lihue,HI,21.97,-159.38
89,17291,17261,7.599956,Kahului Airport,20.89,-156.44,Kahului Airport,HI,20.89,-156.44
92,17294,17292,6.996286,Schofield Barracks,21.49,-158.09,Schofield Barracks,HI,21.49,-158.09
